**Author:** Beatrice Occhiena s314971. See [`LICENSE`](https://github.com/beatrice-occhiena/Computational_intelligence/blob/main/LICENSE) for details.
- institutional email: `S314971@studenti.polito.it`
- personal email: `beatrice.occhiena@live.it`
- github repository: [https://github.com/beatrice-occhiena/Computational_intelligence.git](https://github.com/beatrice-occhiena/Computational_intelligence.git)

**Resources:** These notes are the result of additional research and analysis of the lecture material presented by Professor Giovanni Squillero for the Computational Intelligence course during the academic year 2023-2024 @ Politecnico di Torino. They are intended to be my attempt to make a personal contribution and to rework the topics covered in the following resources.
- [https://github.com/squillero/computational-intelligence](https://github.com/squillero/computational-intelligence)
- Stuart Russel, Peter Norvig, *Artificial Intelligence: A Modern Approach* [3th edition]

.

.

# Lab 2: Nim game

The game of Nim is a well-known **combinatorial misère game** in which we have a number of objects (say coins 🪙 in this case) arranged in different rows. In each turn, a player can remove one or more objects from a single row and the player who takes the last object loses. The game is usually played with a number of rows and a maximum number of objects that can be removed in a turn. 

| Row | Objects |
| --- | --- |
| 0 | 🪙 |
| 1 | 🪙 🪙 🪙 |
| 2 | 🪙 🪙 🪙 🪙 🪙 |


### Task

Our goal is to write an agent able to play Nim with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn. Consider these types of agents:
1. An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
2. An agent using evolved rules using ES

### Instructions
* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)


In [159]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
import numpy as np

## Nim class
First of all, we need to create a class that represents the game of Nim. This class will be used by the agents to play the game. Its attributes are:
- `rows`: the number of rows of the game
- `k`: the maximum number of objects that can be removed in a turn

Each move is represented by the tuple **Nimply** = `(row, num_objects)` and it is applied to the game by the method `nimming`.

In [160]:
# Define a named tuple to represent a move
Nimply = namedtuple("Nimply", "row, num_objects")

class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        """ Initialize the Nim game with num_rows, each row has 2i+1 objects """
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        """ Return True if there are still objects left => we can still play """
        return sum(self._rows) > 0

    def __str__(self):
        """ Return a string representation of the game state """
        str = ""
        for i, row in enumerate(self._rows):
            str += f"Row {i}: " + "🪙" * row + "\n"
        return str

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)
    
    @property
    def k(self) -> int:
        return self._k
    
    def non_empty_rows(self) -> list:
        """ Return a list of non-empty rows indices """
        return [i for i, row in enumerate(self._rows) if row > 0]

    def nimming(self, ply: Nimply) -> None:
        """ Apply the move and update the game state """
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

## Task 1: nim-sum expert agent

### Nim-sum
At any state of the game, we can define the **nim-sum** as the cumulative XOR value of the number of coins in each row. 

| Row | Objects | Binary |
| --- | --- | --- |
| 0 | 🪙 | 001 |
| 1 | 🪙 🪙 🪙 | 011 |
| 2 | 🪙 🪙 🪙 🪙 🪙 | 111 |

`nim-sum` = 001 XOR 011 XOR 111 = 101 = 5

In [161]:
def nim_sum(state: Nim) -> int:

    # Convert the number of objects in each row to binary representation
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])

    # Perform bitwise XOR operation along the columns (axis=0)
    xor = tmp.sum(axis=0) % 2

    # Convert the binary representation back to integer
    return int("".join(str(_) for _ in xor), base=2)

### Winning strategy

#### [1] - Initial phase
If the nim-sum is non-zero and the player to move is playing optimally, then he's guaranteed to win. 
- `nim-sum` = 0 $\implies$ losing position
- `nim-sum` != 0 $\implies$ winning position

> Therefore, the optimal strategy is to always leave the nim-sum equal to zero for the opponent.

e.g. Nimply = (2, 3) applied to the previous game

| Row | Objects | Binary |
| --- | --- | --- |
| 0 | 🪙 | 001 |
| 1 | 🪙 🪙 🪙 | 011 |
| 2 | 🪙 🪙 | 010 |

`nim-sum` = 001 XOR 011 XOR 010 = 000 = 0

> For any state with `nim-sum != 0` there could be multiple optimal moves, but we just need to find one to be sure to win. However, me must pay attention to the case in which there

I implemented the following algorithm:
1. Compute the `nim-sum` of the current state
2. Find a row such that `nim-sum` XOR `num-objects` < `num-objects` (i.e. there are enough objects to remove to make the `nim-sum` equal to zero)
3. Return the Nimply = (row, `num-objects` - (`nim-sum` XOR `num-objects`))

> ⚠️ The k constraint complicates the situation. We must be sure that the number of objects to remove is less than k. If it is not, we must avoid to put the opponent in a favourable position.
- I would like to remove 7 coins, but k = 3 $\implies$ `I can safely remove 3 coins`, since in the next turn the opponent cannot reach the nim-sum = 0 as well
- I would like to remove 5 coins, but k = 3 $\implies$ `I can safely remove 5-3-1 = 1 coins`
- I would like to remove 4 coins, but k = 3 $\implies$ I cannot safely remove any number of coins from this row, and it's `better to choose another one`

In [162]:
def compromise_for_k(ideal_num: int, k: int) -> int:
    if ideal_num - k > k:
        return k
    elif ideal_num - k > 1:
        compromise_val = ideal_num - k - 1
        return compromise_val
    else:
        return -1

In [163]:
def optimal_move(state: Nim) -> Nimply:
    """ Return the optimal move to play from the current state """

    # 1. Compute the nim sum of the current state
    curr_nim_sum = nim_sum(state)

    # 2. If the nim sum is 0, we are in a losing state :/
    # => Find the first non-empty row and remove 1 object
    #    We want the game to last as long as possible, in the hope that the opponent makes a mistake
    if curr_nim_sum == 0:
        for row, num_objects in enumerate(state.rows):
            if num_objects > 0:
                return Nimply(row, 1)
    
    # 3. Otherwise, we find a row and a value that will make the nim sum 0
    # And the win is guaranteed! ;)
    spicy_rows = []
    safe_rows = []
    for row, num_objects in enumerate(state.rows):
        xor_value = curr_nim_sum ^ num_objects
        if xor_value < num_objects:
            spicy_rows.append((row, num_objects - xor_value))
            if state.k is None or num_objects - xor_value <= state.k:
                return Nimply(row, num_objects - xor_value)
        elif num_objects > 0:
            safe_rows.append((row, 1))
    
    # 4. But wait :O ! If we reach here, I cannot set the nim sum to 0 due to the k constraint
    # => We need to avoid to put the opponent in a favorable position
    for row, xor_value in spicy_rows:
        compromised_val = compromise_for_k(xor_value, state.k)
        if compromised_val != -1:
            return Nimply(row, compromised_val)
    return Nimply(*random.choice(safe_rows))

#### [2] - Misère State
When the game is near the end, we must pay attention to the case in which there is only one row with more than one object. 

e.g. Nimply = (1, 2) applied to the previous game

| Row | Objects | Binary |
| --- | --- | --- |
| 0 | 🪙 | 001 |
| 1 | 🪙 | 001 |
| 2 | 🪙 🪙 | 010 |

`nim-sum` = 001 XOR 001 XOR 010 = 010 = 2

> ⚠️ In this case, Choosing to set the next `nim-sum` to 0 would be a losing move! Then, we need to change our optimal strategy, exploiting the fact that a player can remove objects from only one row at a time.

1. The number of remaining rows is even $\implies$ we want to remove entirely the abundant row
2. The number of remaining rows is odd $\implies$ we want to leave only one object in the abundant row

> ⚠️ Once again, the k constraint complicates the situation.

In [164]:
def check_misere(state: Nim) -> int:
    """ Check if the current state is a misere state """
    num_abundant_rows = 0
    abundant_row_index = -1
    for row, num_objects in enumerate(state.rows):
        if num_objects > 1:
            num_abundant_rows += 1
            abundant_row_index = row
    
    # If there is only one row with more than 1 object, we are in a misere state
    if num_abundant_rows == 1:
        return abundant_row_index
    
    # Otherwise, we can proceed to the normal play
    return -1

def optimal_misere_move(state: Nim, abundant_row_index: int) -> Nimply:
    """ Return the optimal move in a misere state """

    num_obj = state.rows[abundant_row_index]

    # 1. The number of remaing rows is even
    # => We want to remove all objects in the abundant row
    if len(state.non_empty_rows()) % 2 == 0:
        if state.k is None or num_obj <= state.k:
            return Nimply(abundant_row_index, num_obj)
        
        # 1.B The k constraint is not satisfied
        compromised_val = compromise_for_k(num_obj, state.k)
        if compromised_val != -1:
            return Nimply(abundant_row_index, compromised_val)
    
    # 2. The number of remaing rows is odd
    # => We want to remove all objects in the abundant row except 1
    else:
        if state.k is None or num_obj-1 <= state.k:
            return Nimply(abundant_row_index, num_obj-1)
        
        # 2.B The k constraint is not satisfied
        compromised_val = compromise_for_k(num_obj-1, state.k)
        if compromised_val != -1:
            return Nimply(abundant_row_index, compromised_val)
    
    # 3. I couldn't find a good compromise
    # => Remove 1 object from any other row
    for row, num_objects in enumerate(state.rows):
        if num_objects > 0 and row != abundant_row_index:
            return Nimply(row, 1)
    
    # 4. In the case only the abundant row is left
    # => Remove 1 object from it
    return Nimply(abundant_row_index, 1)
    

### Agent implementation
Let's now build our expert agent! It is guaranteed to win if the initial state is a winning position. 😎

In [165]:
def expert_agent(state:Nim) -> Nimply:
    """ Alwayes choose the optimal move for the current state """
    # 1. Check if we are in a misere state
    abundant_row_index = check_misere(state)
    if abundant_row_index != -1:
        return optimal_misere_move(state, abundant_row_index)
    
    # 2. Otherwise, we are in a normal state
    return optimal_move(state)

## Other agents
We don't want our expert agent to feel lonely, so we will create other agents to play with it. In this way we can test its performance and see if it is really an expert or not.

In [166]:
def random_agent(state: Nim) -> Nimply:
    """ Alwayes choose a random move for the current state """
    non_empty_rows = state.non_empty_rows()
    row = random.choice(non_empty_rows)
    if state.k is None:
        num_objects = random.randint(1, state.rows[row])
    else:
        num_objects = random.randint(1, min(state.k, state.rows[row]))
    return Nimply(row, num_objects)

In [167]:
def sloppy_agent(state: Nim, distraction_rate = 0.1) -> Nimply:
    """ Choose an optimal move with probability 1-distraction_rate, otherwise choose a random move """
    if random.random() < distraction_rate:
        return random_agent(state)
    return expert_agent(state)

In [168]:
def silly_agent(state: Nim) -> Nimply:
    """ Alwayes choose to take all the objects (or k) in the min objects row """
    non_empty_rows = state.non_empty_rows()
    min_row = min(non_empty_rows, key=lambda x: state.rows[x])
    if state.k is None:
        return Nimply(min_row, state.rows[min_row])
    else:
        return Nimply(min_row, min(state.k, state.rows[min_row]))

### Simple match
Let's now create the code to test the performance of our agents! If we set:
- the Nim state to a winning position for the expert agent
- `k = None` (i.e. no constraint on the number of objects that can be removed in a turn)

The expert agent should always win 🏋🏻

In [213]:
agents = [random_agent, expert_agent]
nim = Nim(4)
print(nim)
player = 0
while nim:
    ply = agents[player](nim)
    print(f"{agents[player].__name__}(player {player}) plays {ply}")
    nim.nimming(ply)
    print(nim)
    player = 1 - player
print(f"{agents[player].__name__}(player {player}) wins!")

Row 0: 🪙
Row 1: 🪙🪙🪙
Row 2: 🪙🪙🪙🪙🪙
Row 3: 🪙🪙🪙🪙🪙🪙🪙

random_agent(player 0) plays Nimply(row=3, num_objects=1)
Row 0: 🪙
Row 1: 🪙🪙🪙
Row 2: 🪙🪙🪙🪙🪙
Row 3: 🪙🪙🪙🪙🪙🪙

expert_agent(player 1) plays Nimply(row=0, num_objects=1)
Row 0: 
Row 1: 🪙🪙🪙
Row 2: 🪙🪙🪙🪙🪙
Row 3: 🪙🪙🪙🪙🪙🪙

random_agent(player 0) plays Nimply(row=1, num_objects=2)
Row 0: 
Row 1: 🪙
Row 2: 🪙🪙🪙🪙🪙
Row 3: 🪙🪙🪙🪙🪙🪙

expert_agent(player 1) plays Nimply(row=3, num_objects=2)
Row 0: 
Row 1: 🪙
Row 2: 🪙🪙🪙🪙🪙
Row 3: 🪙🪙🪙🪙

random_agent(player 0) plays Nimply(row=2, num_objects=4)
Row 0: 
Row 1: 🪙
Row 2: 🪙
Row 3: 🪙🪙🪙🪙

expert_agent(player 1) plays Nimply(row=3, num_objects=3)
Row 0: 
Row 1: 🪙
Row 2: 🪙
Row 3: 🪙

random_agent(player 0) plays Nimply(row=3, num_objects=1)
Row 0: 
Row 1: 🪙
Row 2: 🪙
Row 3: 

expert_agent(player 1) plays Nimply(row=1, num_objects=1)
Row 0: 
Row 1: 
Row 2: 🪙
Row 3: 

random_agent(player 0) plays Nimply(row=2, num_objects=1)
Row 0: 
Row 1: 
Row 2: 
Row 3: 

expert_agent(player 1) wins!


## Task 2: ES agent
For this task, I want create an agent that uses ES to learn the optimal strategy to play Nim. The main idea is to create a population of agents, each one with a different set of weighted rules, and evolve them over time. To test their performance, they will play against the previously created agents.

### Rule representation
I designed a generic rule as a combination of the following elements:
- ❓ `condition` = it evaluates certain aspects of the game state
- 📍 `position` = it indicates the row to which the action must be applied
- 🫱🏻 `action` = it indicates the number of objects to remove from the row

In [282]:

Rule = namedtuple("Rule", ["condition", "position", "action"])
Rule.__str__ = lambda self: f"{self.condition.__name__} then {self.action.__name__} at {self.position.__name__}"

#### Conditions

In [283]:
def if_odd_rows(state: Nim) -> bool:
    return len(state.non_empty_rows()) % 2 == 1

def if_even_rows(state: Nim) -> bool:
    return len(state.non_empty_rows()) % 2 == 0

def if_misere(state: Nim) -> bool:
    return check_misere(state) != -1

def if_nimsum_zero(state: Nim) -> bool:
    return nim_sum(state) == 0

def if_nimsum_not_zero(state: Nim) -> bool:
    return nim_sum(state) != 0

conditions = [if_odd_rows, if_even_rows, if_misere, if_nimsum_zero, if_nimsum_not_zero]

#### Positions

In [284]:
def min_position(state: Nim) -> int:
    return min(state.non_empty_rows(), key=lambda x: state.rows[x])

def max_position(state: Nim) -> int:
    return max(state.non_empty_rows(), key=lambda x: state.rows[x])

def random_position(state: Nim) -> int:
    return random.choice(state.non_empty_rows())

# BEANCHMARK
def optimal_position(state: Nim) -> int:
    return expert_agent(state).row

positions = [min_position, max_position, random_position]

#### Actions

In [285]:
def get_one(state: Nim, row: int) -> Nimply:
    return Nimply(row, 1)

def get_all_but_one(state: Nim, row: int) -> Nimply:
    if state.k is None:
        return Nimply(row, state.rows[row] - 1)
    else:
        return Nimply(row, min(state.k, state.rows[row] - 1))

def get_max(state: Nim, row: int) -> Nimply:
    if state.k is None:
        return Nimply(row, state.rows[row])
    else:
        return Nimply(row, min(state.k, state.rows[row]))
    
def get_random(state: Nim, row: int) -> Nimply:
    if state.k is None:
        return Nimply(row, random.randint(1, state.rows[row]))
    else:
        return Nimply(row, random.randint(1, min(state.k, state.rows[row])))

# BENCHMARK
def get_optimal(state: Nim, row: int) -> Nimply:
    return expert_agent(state)

actions = [get_one, get_all_but_one, get_max, get_random]

### Rules generation
To explore a wide range of strategies, we want to generate all possible combinations of conditions, positions, and actions. This exhaustive approach allows us to experiment with various rule sets and observe how different combinations impact the agent's performance.

> To check if the ES is working properly, we can insert a special set of **advanced rules** `directly derived from the expert agent`. In this way, we can see if the ES is able to maximize the weights of these rules and to learn the optimal strategy after a certain number of generations.

In [286]:
def generate_all_possible_rules() -> list:
    rules = []
    for condition in conditions:
        for position in positions:
            for action in actions:
                rules.append(Rule(condition, position, action))
        # BENCHMARK
        rules.append(Rule(condition, optimal_position, get_optimal))
    return rules

### ES individuals
The ESIndividual class represents an individual within the Evolutionary Strategy (ES) framework for the Nim game. Each individual is characterized by:
- 📃 `rules`: a list of all possible rules
- 🎚️ `weights`: a list of weights corresponding to the importance of selecting each rule
- 📏 `sigmas`: a list of standard deviations corresponding to self-adapt the weights
- 🏋🏻 `fitness`: a measure of the individual's performance

#### Rule selection
The pick_rule method selects a rule based on the weights and conditions. It considers rules whose conditions hold true for the current game state, computes the probability of each rule, and selects one accordingly.

> 🛞 This method is inspired by the roulette wheel selection method used in genetic algorithms.

#### Mutation
The mutate method applies a mutation to the individual's weights and sigmas, with a certain probability. The mutation is a Gaussian random number with `mean 0` and standard deviation `sigma[i]`.

#### Fitness evaluation
To evaluate the fitness of an individual, we will make it play a certain number of games against other agents. The fitness is the proportion of wins in these games.

> ⚠️ We must pay attention to the case in which the agent plays against the expert agent. In this case, for the metric to be meaningful, we must make sure that the expert agent is not starting the game in a winning position.

In [287]:
class ESIndividual:
    def __init__(self) -> None:
        self.rules = generate_all_possible_rules()
        self.weights = np.random.uniform(0, 1, len(self.rules))
        self.sigmas = np.random.uniform(0, 3, len(self.rules))
        self.fitness = 0
    
    def __str__(self) -> str:
        str = ""
        # show the fitness
        str += f"🏋🏻 Fitness: {self.fitness}\n"
        # show 10 top rules with their respective weights
        top_10_indices = np.argsort(self.weights)[-10:]
        for i in top_10_indices:
            str += f"\t🎚️ WEIGHT: {self.weights[i]:.2f}   📃 RULE: {self.rules[i]}\t \n"
        return str
    
    def pick_rule(self, state: Nim) -> Rule:
        """ Pick a rule according to the weights and the conditions """

        # 1. Select the rules in which the condition is true for the current state
        indices = [i for i, rule in enumerate(self.rules) if rule.condition(state)]
        valid_rules = [self.rules[i] for i in indices]

        # 2. Compute the probability of each rule
        probabilities = self.weights[indices] / self.weights[indices].sum()

        # 3. Pick a rule according to the probabilities
        index = np.random.choice(len(valid_rules), p=probabilities)
        rule = valid_rules[index]
        return rule

    
    def play(self, state: Nim) -> Nimply:
        """ Play a move according to the rules """
        rule = self.pick_rule(state)
        return rule.action(state, rule.position(state))
    
    def evaluate(self, num_games: int = 60) -> None:
        """ Evaluate the fitness of the individual by playing num_games against different agents """
        wins = 0

        # 1. Play against the expert agent
        players = [expert_agent, self.play]
        for _ in range(num_games//2):
            nim = Nim(4) # expert in a loosing state
            player = 0
            while nim:
                ply = players[player](nim)
                nim.nimming(ply)
                player = 1 - player
            wins += player # if player == 1, then the individual wins

        # 2. Play against the random agent
        players = [random_agent, self.play]
        for _ in range(num_games//2):
            nim = Nim(4)
            player = 0
            while nim:
                ply = players[player](nim)
                nim.nimming(ply)
                player = 1 - player
            wins += player

        # 3. Update the fitness
        self.fitness = wins / num_games

    def mutate(self, mutation_rate: float = 0.1) -> None:
        """ Mutate the individual by adding a random value to each weight """
        for i in range(len(self.weights)):
            if random.random() < mutation_rate:
                self.weights[i] += random.gauss(0, self.sigmas[i])
                self.weights[i] = max(0, self.weights[i]) # clip the weights to be positive
    
    def self_adaptat(self) -> None:
        """ 
            Adapt the sigmas according to the win rate (i.e. the fitness)
            - high num of wins: I don't want to change the sigmas too much (exploitation)
            - low num: I want to significantly change the sigmas (exploration)
        """
        win_rate = self.fitness
        # Self-adaptation of the sigmas
        for i in range(len(self.sigmas)):
            if random.random() > win_rate:
                self.sigmas[i] += random.gauss(0, 1 - win_rate)
                self.sigmas[i] = max(0, self.sigmas[i])

### ($\mu/\rho$ , $\lambda$)-ES
At each generation, we will select the best $\mu$ individuals and generate $\lambda$ new ones. The new individuals are generated by mutating the weights and sigmas of the best individuals. The $\mu$ best individuals are selected based on their fitness.

For each generation, I printed the following information:
- the best individual
- its fitness
- its top 10 rules with the highest weights


In [288]:
# PARAMETERS
num_parents = 10    # mu - population size
num_children = 20  # lambda - number of children
num_generations = 20
mutation_rate = 0.5
crossover_rate = 0.5

In [289]:
# 1. Initialize the population
population = [ESIndividual() for _ in range(num_children)]

# 2. Evaluate the fitness of each individual
for individual in population:
    individual.evaluate()

# 3. Sort the population according to the fitness
population.sort(key=lambda x: x.fitness, reverse=True)

# 4. Print the best individual
print(population[0])

# 5. Repeat for num_generations
for generation in range(num_generations):
    print(f"Generation {generation+1}")
    # 5.1. Select the parents
    parents = population[:num_parents]

    # 5.2. Generate the children
    children = []
    for _ in range(num_children):
        # 5.2.1. Select two parents
        parent1, parent2 = random.choices(parents, k=2)

        # 5.2.2. Crossover
        child = deepcopy(parent1)
        for i in range(len(child.weights)):
            if random.random() < 0.5:
                child.weights[i] = parent2.weights[i]
                child.sigmas[i] = parent2.sigmas[i]

        # 5.2.3. Mutate
        child.mutate(mutation_rate)

        # 5.2.4. Evaluate
        child.evaluate()

        # 5.2.5. Add to the children
        children.append(child)

    # 5.3. Select the survivors
    population = parents + children
    population.sort(key=lambda x: x.fitness, reverse=True)
    population = population[:num_parents]

    # 5.4. Self-adaptation
    for individual in population:
        individual.self_adaptat()

    # 5.5. Print the best individual
    print(population[0])

🏋🏻 Fitness: 0.5166666666666667
	🎚️ WEIGHT: 0.88   📃 RULE: if_nimsum_zero then get_max at max_position	 
	🎚️ WEIGHT: 0.90   📃 RULE: if_misere then get_max at min_position	 
	🎚️ WEIGHT: 0.90   📃 RULE: if_nimsum_zero then get_all_but_one at random_position	 
	🎚️ WEIGHT: 0.92   📃 RULE: if_nimsum_not_zero then get_max at min_position	 
	🎚️ WEIGHT: 0.93   📃 RULE: if_nimsum_not_zero then get_random at random_position	 
	🎚️ WEIGHT: 0.93   📃 RULE: if_misere then get_one at max_position	 
	🎚️ WEIGHT: 0.94   📃 RULE: if_misere then get_max at random_position	 
	🎚️ WEIGHT: 0.97   📃 RULE: if_nimsum_zero then get_max at min_position	 
	🎚️ WEIGHT: 0.97   📃 RULE: if_nimsum_zero then get_optimal at optimal_position	 
	🎚️ WEIGHT: 0.99   📃 RULE: if_nimsum_not_zero then get_random at max_position	 

Generation 1
🏋🏻 Fitness: 0.6833333333333333
	🎚️ WEIGHT: 0.93   📃 RULE: if_odd_rows then get_max at random_position	 
	🎚️ WEIGHT: 0.95   📃 RULE: if_nimsum_not_zero then get_max at min_position	 
	🎚️ WEIGHT: 0.95

# TO-DO
- [ ] Correct the action of 'TAKING 0 OBJECTS' (when the condition allows it e.g. all but one with only one object left)
- [ ] Create a method for crossover
- [ ] Create a sub-method for evaluation